<a href="https://colab.research.google.com/github/Natural-State/agol-data-workflows/blob/master/code/Colab%20notebooks/20_GEDI_L2A_L2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extract canopy metrics measure from GEDI Level 2A and 2B platform

## Import gee and authenticate

In [ ]:
import ee

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

## Input arguments for data extraction

In [ ]:
# Area of interest
aoi = ee.FeatureCollection("projects/ns-agol-rs-data/assets/MKR")
aoi_name = "MKR"


# GEE layer ID - Level 2A
layer_name = "RS_063" # rh100

# GEE layer ID - Level 2B
layer_dict = {
    "cover": "RS_064",
    "pai": "RS_065",
    "pavd_z25": "RS_066"
}



## GEDI L2A



### Process

In [ ]:
gedi2A = ee.ImageCollection("LARSE/GEDI/GEDI02_A_002_MONTHLY")

def qualityMask(image):
    return image.updateMask(image.select('quality_flag').eq(1)).updateMask(image.select('degrade_flag').eq(0))

canopy_height = gedi2A \
             .map(qualityMask) \
             .select("rh100") \
             .mean() \
             .clip(aoi)

print(gedi2A.size().getInfo())
print(canopy_height.getInfo())

### Export

In [ ]:
task1 = ee.batch.Export.image.toDrive(image = canopy_height,
                                     region = aoi.geometry(),
                                     description = "EXPORT IMAGE TO DRIVE",
                                     folder = "GEE_exports",
                                     fileNamePrefix = layer_name + "_" + aoi_name,
                                     scale = 25,
                                     maxPixels = 10e12,
                                     crs = "EPSG:4326"
                                     )
task1.start()

In [ ]:
task_id = task1.status()['id']
task_state = task1.status()['state']
print(task_id, task_state)

## GEDI L2B

### Process

In [ ]:
gedi2B = ee.ImageCollection('LARSE/GEDI/GEDI02_B_002_MONTHLY')

def qualityMask(image):
    return image.updateMask(image.select('l2b_quality_flag').eq(1)).updateMask(image.select('degrade_flag').eq(0))

def gedi_image(metric):
    return gedi2B \
         .map(qualityMask) \
         .select(metric) \
         .mean() \
         .clip(aoi) \
         .set({"canopy_metric": metric})

cover = gedi_image("cover")
pai = gedi_image("pai")
pavd_z25 = gedi_image("pavd_z25")

print(cover.getInfo())
print(cover.get("canopy_metric").getInfo())

print(pavd_z25.getInfo())
print(pavd_z25.get("canopy_metric").getInfo())

### Export

In [ ]:
output_name = f"{layer_dict[cover.get('canopy_metric').getInfo()]}_{aoi_name}"
print(output_name)

task2 = ee.batch.Export.image.toDrive(image = cover,
                                     region = aoi.geometry(),
                                     description = "EXPORT IMAGE TO DRIVE",
                                     folder = "GEE_exports",
                                     fileNamePrefix = output_name,
                                     scale = 25,
                                     maxPixels = 10e12,
                                     crs = "EPSG:4326"
                                     )
task2.start()

In [ ]:
task_id = task2.status()['id']
task_state = task2.status()['state']
print(task_id, task_state)

In [ ]:
output_name = f"{layer_dict[pai.get('canopy_metric').getInfo()]}_{aoi_name}"
print(output_name)

task3 = ee.batch.Export.image.toDrive(image = pai,
                                     region = aoi.geometry(),
                                     description = "EXPORT IMAGE TO DRIVE",
                                     folder = "GEE_exports",
                                     fileNamePrefix = output_name,
                                     scale = 25,
                                     maxPixels = 10e12,
                                     crs = "EPSG:4326"
                                     )
task3.start()

In [ ]:
output_name = f"{layer_dict[pavd_z25.get('canopy_metric').getInfo()]}_{aoi_name}"
print(output_name)

task4 = ee.batch.Export.image.toDrive(image = pavd_z25,
                                     region = aoi.geometry(),
                                     description = "EXPORT IMAGE TO DRIVE",
                                     folder = "GEE_exports",
                                     fileNamePrefix = output_name,
                                     scale = 25,
                                     maxPixels = 10e12,
                                     crs = "EPSG:4326"
                                     )
task4.start()

In [ ]:
task_id = task4.status()['id']
task_state = task4.status()['state']
print(task_id, task_state)